<a href="https://colab.research.google.com/github/JhonnyLimachi/IA_en_Salud/blob/main/5_Regresi%C3%B3n_log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 4.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Regresión logística

En el ejemplo de regresión lineal, queríamos inferir qué precio de venta poner a una casa, dado un conjunto de *características*. En otras palabras, queríamos predecir un valor continuo.

En cambio, la regresión logística se utiliza para clasificar algo dentro de un conjunto de categorías. Como ejemplo, podemos tomar el *conjunto de datos* de pacientes con enfermedad cardiovascular utilizado en el módulo anterior.

<p align=center><img src="https://1394217531-files.gitbook.io/~/files/v0/b/gitbook-legacy-files/o/assets%2F-LvBP1svpACTB1R1x_U4%2F-Lw70vAIGPfRR1AjprLi%2F-LwAVc1EdfmPMge5dlYC%2Fimage. png?alt=media&token=d72e3231-0d64-4bb7-9e4c-20577940763d" width="60%"></p>

En la última línea, ¿cómo clasificaríamos al paciente como enfermo del corazón o no? La regresión lineal no es adecuada en este caso, ya que trabaja con valores continuos.

La regresión logística es ideal en este tipo de situaciones de clasificación, ya que trabaja con probabilidades. En este caso, tras entrenar el modelo seríamos capaces de ver la probabilidad de que un determinado paciente tenga un problema cardíaco o no.

<p align=center><img src="https://media5.datahacker.rs/2021/01/83.jpg" width="50%"></p>

Piensa en la imagen anterior como si el eje x fuera la edad de un paciente y el eje y su probabilidad de tener una cardiopatía.

Como puede ver, necesitamos asignar valores a un rango $[0,1]$. Para ello, utilizamos la función sigmoide,

$$
sigmoide(z) = \frac{1}{1 + e^{-z}}
$$

donde $z= b + w_1x_1 + w_2x_2 + \dots + w_mx_m$ es la *salida* de la capa lineal de un modelo de regresión logística. La curva **sigmoidal** mapeará este valor (cualquiera) al itnerval entre 0 y 1.

Al igual que en la regresión lineal teníamos el error al cuadrado (*pérdida al cuadrado*), la *función de pérdida* para la regresión logística se representa mediante

$$
\text{Log Loss} = \sum_{(x, y) \in D} -y\log(\hat{y}) - (1 -y)\log(1-\hat{y})
$$



## Regresión Logística en `scikit-learn`

El conjunto de datos original, que ha sido adaptado en este *notebook*, fue recogido de la ***Cleveland Clinic Foundation*** y está disponible en el conocido [UCI *Machine Learning* Repository](https://archive.ics.uci.edu/ml/datasets/heart+Disease).

A diferencia del fichero `csv` que se puso a disposición en el Módulo 3, [en este enlace](http://dl.dropboxusercontent.com/s/6d91j46mkcdj4qv/heart-disease-clean.csv?dl=0) puedes encontrar el fichero con los datos ya procesados. Para nuestra variable objetivo (*target*) encontrará las siguientes categorías:

* `num` - diagnóstico de cardiopatía (diagnóstico angiográfico)
  * 0: no hay cardiopatía (< 50% de estrechamiento del diámetro)
  * 1: hay cardiopatía (> 50% de estrechamiento del diámetro)